In [115]:
# coding=utf-8
import pandas as pd
import csv  # Import csv
import tweepy
import re
import string
import nltk
import pandas as pd
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
pd.set_option('display.max_colwidth',1000)

In [3]:
#col_list = ["ID", "FULL TEXT","CREATED AT","LABEL"]
#data = pd.read_csv('get_tweets.csv', usecols=col_list)
#data.columns = ['ID','FULL_TEXT','CREATED_AT','LABEL']
#print(data)

In [4]:
################ working code#############################
#data = pd.read_csv('TweetsWithLabels', header = None, encoding='utf-8')
#data.columns = ['ID','User','FULL_TEXT','CREATED_AT','LABEL']
#data = data.dropna()
#data.head()
    

In [26]:

data = pd.read_csv('get_tweets.csv', header = None, encoding='utf-8', engine='python')
data.columns = ['ID','User','FULL_TEXT','CREATED_AT','LABEL']
data = data.dropna()
data.head()

,ID,User,FULL_TEXT,CREATED_AT,LABEL
2,0.0,1.41416E+18,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්‍රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප‍්‍රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද (11) අත්අඩංගුවට ගනු ලැබ තිබේ.\r\n#arrestrapists #lka,7/11/2021 9:59,Crime
4,1.0,1.41415E+18,ක්‍රීඩා අමාත්‍යාංශයෙන් බොක්සිං වෙත මිලියන 50ක්! https://t.co/27rZZNdJoV via @ThePapareSports,7/11/2021 9:14,Sport
6,2.0,1.41415E+18,මේසපන්දුව දියුණු කරන්නට ක්‍රීඩා අමාත්‍යංශය ගිවිසුම්ගත වෙයි https://t.co/8shOPe5gul via @ThePapareSports,7/11/2021 9:13,Sport
8,3.0,1.41411E+18,"1992/93 පී. සරවනමුත්තු කුසලාන අන්තර්-සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්‍රීඩා සමාජ පිල.\r\n\r\nමෙහි ශ්‍රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්‍රීඩකයන් රැසක් සිටින අතර, එවක ශ්‍රී ලංකා නායක අර්ජුන රණතුංග සාමාන්‍ය ක්‍රීඩකයෙක් වශයෙන් එම වසරේදී ක්‍රීඩා සමාජයට ක්‍රීඩා කර ඇත. https://t.co/hzZv1mVKSR",7/11/2021 6:39,Sport
10,4.0,1.41411E+18,“ක්‍රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන නෑ” සුනිල් ගුණවර්ධන https://t.co/jbNn6kMzEW #slathletics #namalrajapaksa @RajapaksaNamal https://t.co/iidrXsiHNe,7/11/2021 6:11,Politics


In [27]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
    tokens = [regex.sub('', w) for w in tokens]
    tokens = " ".join([c for c in tokens if c not in string.punctuation])
    return tokens

In [28]:
data['cleaned_msg'] = data['FULL_TEXT'].apply(lambda x: clean_text(x))
data.head()

,ID,User,FULL_TEXT,CREATED_AT,LABEL,cleaned_msg
2,0.0,1.41416E+18,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්‍රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප‍්‍රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද (11) අත්අඩංගුවට ගනු ලැබ තිබේ.\r\n#arrestrapists #lka,7/11/2021 9:59,Crime,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප්රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද අත්අඩංගුවට ගනු ලැබ තිබේ
4,1.0,1.41415E+18,ක්‍රීඩා අමාත්‍යාංශයෙන් බොක්සිං වෙත මිලියන 50ක්! https://t.co/27rZZNdJoV via @ThePapareSports,7/11/2021 9:14,Sport,ක්රීඩා අමාත්යාංශයෙන් බොක්සිං වෙත මිලියන ක්
6,2.0,1.41415E+18,මේසපන්දුව දියුණු කරන්නට ක්‍රීඩා අමාත්‍යංශය ගිවිසුම්ගත වෙයි https://t.co/8shOPe5gul via @ThePapareSports,7/11/2021 9:13,Sport,මේසපන්දුව දියුණු කරන්නට ක්රීඩා අමාත්යංශය ගිවිසුම්ගත වෙයි
8,3.0,1.41411E+18,"1992/93 පී. සරවනමුත්තු කුසලාන අන්තර්-සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්‍රීඩා සමාජ පිල.\r\n\r\nමෙහි ශ්‍රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්‍රීඩකයන් රැසක් සිටින අතර, එවක ශ්‍රී ලංකා නායක අර්ජුන රණතුංග සාමාන්‍ය ක්‍රීඩකයෙක් වශයෙන් එම වසරේදී ක්‍රීඩා සමාජයට ක්‍රීඩා කර ඇත. https://t.co/hzZv1mVKSR",7/11/2021 6:39,Sport,පී සරවනමුත්තු කුසලාන අන්තර්සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්රීඩා සමාජ පිල මෙහි ශ්රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්රීඩකයන් රැසක් සිටින අතර එවක ශ්රී ලංකා නායක අර්ජුන රණතුංග සාමාන්ය ක්රීඩකයෙක් වශයෙන් එම වසරේදී ක්රීඩා සමාජයට ක්රීඩා කර ඇත
10,4.0,1.41411E+18,“ක්‍රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන නෑ” සුනිල් ගුණවර්ධන https://t.co/jbNn6kMzEW #slathletics #namalrajapaksa @RajapaksaNamal https://t.co/iidrXsiHNe,7/11/2021 6:11,Politics,ක්රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන නෑ සුනිල් ගුණවර්ධන


In [29]:
#######################c Remove Stop words Method 1: Remove stop words  #########################################

In [30]:
cleaned_data = data['cleaned_msg']
wli = []
for l in cleaned_data:
    l = l.split()
    
    for w in l:
        wli.append(w)


In [31]:
# create word dictionary word and word count
wli.sort()
wd = {} 
for w in wli:
    wd[w] = wli.count(w)
    
print('\n{:^8}{:^8}'.format('word','count'))
for w in wd:
    print('{:^8}{:^8}'.format(w,wd[w]))
    
print("")
    


  word   count  
  අංක      1    
 අංශයේ     1    
  අකල්     1    
  අක්ක     1    
 අක්කා     1    
   අග      1    
 අගමැති    3    
අගමැතිධූරය   1    
අග්රාමාත්ය   3    
 අගාධය     1    
  අටර්     1    
   අඩ      1    
අඩංගුවට    3    
අඩස්සියේ   1    
  අඩාල     1    
  අඩු      3    
අඩුගානේ    1    
 අඩුයි     1    
අතකොලුඑයාලවත්   1    
 අතකොළු    1    
  අතට      1    
  අතර      3    
  අතරය     1    
 අතරින්    1    
  අත්      3    
අත්අඩංගුවට   5    
අත්තනෝමතික   1    
අත්දැකීම්   2    
අත්යවශ්යයි   1    
 අත්සන්    1    
අත්හිටුවමින්   2    
අත්හිටුවයි   7    
අත්හිටුවීමට   1    
අතිවිශේෂ   2    
  අතීත     1    
 අතීතය     1    
  අතේ      1    
   අද      9    
 අදහස්     2    
  අදාල     1    
අදිකරනයෙ   1    
අධ්යක්ශණය   1    
අධ්යක්ෂණය   2    
අධ්යක්ෂණයට   1    
අධ්යක්ෂිකා   1    
 අධ්යන     1    
අධ්යයනාංශ   1    
අධ්යාපන    2    
අධ්යාපනය   3    
අධිකාරිය   1    
අනතුරුව    1    
  අන්ත     1    
 අන්තයේ    1    
අන්තර්ජාලය   2    
අන්තර්ජාලයට   1    
අන්තර්සමාජ

In [32]:
stopword = []
for w in wd:
    if(wd[w]>10):
        stopword.append(w)
print(stopword)

['ඒ', 'කරන', 'ක්රීඩා', 'තමයි', 'ද', 'දේශපාලන', 'නම්', 'නෑ', 'පක්ෂ', 'මම', 'මේ', 'රට', 'රටේ', 'වෙනුවෙන්', 'ශ්රී', 'සහ', 'හා']


In [33]:
# def clean_text(text):
#     tokens = nltk.word_tokenize(text)
#     tokens = " ".join([word for word in tokens if word not in stopword])
#     return tokens

In [34]:
# data['remove_stopwords'] = data['cleaned_msg'].apply(lambda x: clean_text(x))
# data.head()

In [35]:
############################ Stop Word removing Method 2 - using predefined stop words ####################################

In [36]:
stopwordsreader = open('StopWords', 'r', encoding='utf-8')
reader = csv.reader(stopwordsreader)

swords = []
for row in reader:
    swords.append(row[0])
print(swords)
    

['සහ', 'සමග', 'සමඟ', 'අහා', 'ආහ්', 'ආ', 'ඕහෝ', 'අනේ', 'අඳෝ', 'අපොයි', 'අපෝ', 'අයියෝ', 'ආයි', 'ඌයි', 'චී', 'චිහ්', 'චික්', 'හෝ\u200d', 'දෝ', 'දෝහෝ', 'මෙන්', 'සේ', 'වැනි', 'බඳු', 'වන්', 'අයුරු', 'අයුරින්', 'ලෙස', 'වැඩි', 'ශ්\u200dරී', 'හා', 'ය', 'නිසා', 'නිසාවෙන්', 'බවට', 'බව', 'බවෙන්', 'නම්', 'වැඩි', 'සිට', 'දී', 'මහා', 'මහ', 'පමණ', 'පමණින්', 'පමන', 'වන', 'විට', 'විටින්', 'මේ', 'මෙලෙස', 'මෙයින්', 'ඇති', 'ලෙස', 'සිදු', 'වශයෙන්', 'යන', 'සඳහා', 'මගින්', 'හෝ\u200d', 'ඉතා', 'ඒ', 'එම', 'ද', 'අතර', 'විසින්', 'සමග', 'පිළිබඳව', 'පිළිබඳ', 'තුළ', 'බව', 'වැනි', 'මහ', 'මෙම', 'මෙහි', 'මේ', 'වෙත', 'වෙතින්', 'වෙතට', 'වෙනුවෙන්', 'වෙනුවට', 'වෙන', 'ගැන', 'නෑ', 'අනුව', 'නව', 'පිළිබඳ', 'විශේෂ', 'දැනට', 'එහෙන්', 'මෙහෙන්', 'එහේ', 'මෙහේ', 'ම', 'තවත්', 'තව ', 'සහ', 'දක්වා', 'ට', 'ගේ', 'එ', 'ක', 'ක්', 'බවත්', 'බවද', 'මත', 'ඇතුලු', 'ඇතුළු', 'මෙසේ', 'වඩා', 'වඩාත්ම', 'නිති', 'නිතිත්', 'නිතොර', 'නිතර', 'ඉක්බිති', 'දැන්', 'යලි', 'පුන', 'ඉතින්', 'සිට', 'සිටන්', 'පටන්', 'තෙක්', 'දක්වා', 'සා', 'තාක්', 'තුවක්', 'පවා', 'ද

In [37]:
def remove_stop_words(text):
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in swords]
    tokens = " ".join([c for c in tokens if c not in string.punctuation])
    return tokens

In [38]:
data['remove_stopword'] = data['cleaned_msg'].apply(lambda x: remove_stop_words(x))
data.head()

,ID,User,FULL_TEXT,CREATED_AT,LABEL,cleaned_msg,remove_stopword
2,0.0,1.41416E+18,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්‍රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප‍්‍රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද (11) අත්අඩංගුවට ගනු ලැබ තිබේ.\r\n#arrestrapists #lka,7/11/2021 9:59,Crime,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප්රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද අත්අඩංගුවට ගනු ලැබ තිබේ,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු රජයේ ප්රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා කාන්තා කාර්යාංශය අද අත්අඩංගුවට ගනු ලැබ තිබේ
4,1.0,1.41415E+18,ක්‍රීඩා අමාත්‍යාංශයෙන් බොක්සිං වෙත මිලියන 50ක්! https://t.co/27rZZNdJoV via @ThePapareSports,7/11/2021 9:14,Sport,ක්රීඩා අමාත්යාංශයෙන් බොක්සිං වෙත මිලියන ක්,ක්රීඩා අමාත්යාංශයෙන් බොක්සිං මිලියන
6,2.0,1.41415E+18,මේසපන්දුව දියුණු කරන්නට ක්‍රීඩා අමාත්‍යංශය ගිවිසුම්ගත වෙයි https://t.co/8shOPe5gul via @ThePapareSports,7/11/2021 9:13,Sport,මේසපන්දුව දියුණු කරන්නට ක්රීඩා අමාත්යංශය ගිවිසුම්ගත වෙයි,මේසපන්දුව දියුණු කරන්නට ක්රීඩා අමාත්යංශය ගිවිසුම්ගත වෙයි
8,3.0,1.41411E+18,"1992/93 පී. සරවනමුත්තු කුසලාන අන්තර්-සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්‍රීඩා සමාජ පිල.\r\n\r\nමෙහි ශ්‍රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්‍රීඩකයන් රැසක් සිටින අතර, එවක ශ්‍රී ලංකා නායක අර්ජුන රණතුංග සාමාන්‍ය ක්‍රීඩකයෙක් වශයෙන් එම වසරේදී ක්‍රීඩා සමාජයට ක්‍රීඩා කර ඇත. https://t.co/hzZv1mVKSR",7/11/2021 6:39,Sport,පී සරවනමුත්තු කුසලාන අන්තර්සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්රීඩා සමාජ පිල මෙහි ශ්රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්රීඩකයන් රැසක් සිටින අතර එවක ශ්රී ලංකා නායක අර්ජුන රණතුංග සාමාන්ය ක්රීඩකයෙක් වශයෙන් එම වසරේදී ක්රීඩා සමාජයට ක්රීඩා කර ඇත,පී සරවනමුත්තු කුසලාන අන්තර්සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්රීඩා සමාජ පිල ශ්රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්රීඩකයන් රැසක් සිටින එවක ශ්රී ලංකා නායක අර්ජුන රණතුංග සාමාන්ය ක්රීඩකයෙක් වසරේදී ක්රීඩා සමාජයට ක්රීඩා කර ඇත
10,4.0,1.41411E+18,“ක්‍රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන නෑ” සුනිල් ගුණවර්ධන https://t.co/jbNn6kMzEW #slathletics #namalrajapaksa @RajapaksaNamal https://t.co/iidrXsiHNe,7/11/2021 6:11,Politics,ක්රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන නෑ සුනිල් ගුණවර්ධන,ක්රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන සුනිල් ගුණවර්ධන


In [39]:
########################################## Stemming ##################################################

In [40]:
stem_dict = open("stem_dictionary.txt", "r", encoding='utf-8')
stem_dict = map(lambda s: s.strip(), stem_dict)

stem_dictionary = {}

for s in stem_dict:
    s = s.split("\t")
    stem_dictionary[s[0]] = s[1]
    


In [41]:
def stemming_words(text):
    tokens = nltk.word_tokenize(text)
    for k, v in enumerate(tokens):
        tokens[k] = stem_dictionary.get(v, v)
    tokens = " ".join([c for c in tokens if c not in string.punctuation])
    return tokens

In [42]:
data['stemming_words'] = data['remove_stopword'].apply(lambda x: stemming_words(x))
data.head()

,ID,User,FULL_TEXT,CREATED_AT,LABEL,cleaned_msg,remove_stopword,stemming_words
2,0.0,1.41416E+18,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්‍රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප‍්‍රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද (11) අත්අඩංගුවට ගනු ලැබ තිබේ.\r\n#arrestrapists #lka,7/11/2021 9:59,Crime,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු සහ රජයේ ප්රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා හා කාන්තා කාර්යාංශය විසින් අද අත්අඩංගුවට ගනු ලැබ තිබේ,අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ සිද්ධිය සම්බන්ධයෙන් පොලිස් ක්රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු රජයේ ප්රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා කාන්තා කාර්යාංශය අද අත්අඩංගුවට ගනු ලැබ තිබේ,අන්තර්ජාල ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජන කිරීමේ සිද්ධිය සම්බන්ධ පොලිස් ක්රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු රජයේ ප්රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා කාන්තා කාර්යාංශ අද අත්අඩංගුව ගනු ලැබ තිබේ
4,1.0,1.41415E+18,ක්‍රීඩා අමාත්‍යාංශයෙන් බොක්සිං වෙත මිලියන 50ක්! https://t.co/27rZZNdJoV via @ThePapareSports,7/11/2021 9:14,Sport,ක්රීඩා අමාත්යාංශයෙන් බොක්සිං වෙත මිලියන ක්,ක්රීඩා අමාත්යාංශයෙන් බොක්සිං මිලියන,ක්රීඩා අමාත්යාංශයෙන් බොක්සිං මිලියන
6,2.0,1.41415E+18,මේසපන්දුව දියුණු කරන්නට ක්‍රීඩා අමාත්‍යංශය ගිවිසුම්ගත වෙයි https://t.co/8shOPe5gul via @ThePapareSports,7/11/2021 9:13,Sport,මේසපන්දුව දියුණු කරන්නට ක්රීඩා අමාත්යංශය ගිවිසුම්ගත වෙයි,මේසපන්දුව දියුණු කරන්නට ක්රීඩා අමාත්යංශය ගිවිසුම්ගත වෙයි,මේසපන්දුව දියුණු කරන්නට ක්රීඩා අමාත්යංශය ගිවිසුම්ගත වෙයි
8,3.0,1.41411E+18,"1992/93 පී. සරවනමුත්තු කුසලාන අන්තර්-සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්‍රීඩා සමාජ පිල.\r\n\r\nමෙහි ශ්‍රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්‍රීඩකයන් රැසක් සිටින අතර, එවක ශ්‍රී ලංකා නායක අර්ජුන රණතුංග සාමාන්‍ය ක්‍රීඩකයෙක් වශයෙන් එම වසරේදී ක්‍රීඩා සමාජයට ක්‍රීඩා කර ඇත. https://t.co/hzZv1mVKSR",7/11/2021 6:39,Sport,පී සරවනමුත්තු කුසලාන අන්තර්සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්රීඩා සමාජ පිල මෙහි ශ්රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්රීඩකයන් රැසක් සිටින අතර එවක ශ්රී ලංකා නායක අර්ජුන රණතුංග සාමාන්ය ක්රීඩකයෙක් වශයෙන් එම වසරේදී ක්රීඩා සමාජයට ක්රීඩා කර ඇත,පී සරවනමුත්තු කුසලාන අන්තර්සමාජ පළමු ඛාණ්ඩයේ ශූරයන් වූ සිංහල ක්රීඩා සමාජ පිල ශ්රී ලංකව විවිධ අවධි වල නියෝජනය කල ක්රීඩකයන් රැසක් සිටින එවක ශ්රී ලංකා නායක අර්ජුන රණතුංග සාමාන්ය ක්රීඩකයෙක් වසරේදී ක්රීඩා සමාජයට ක්රීඩා කර ඇත,පී සරවනමුත්තු කුසලා අන්තර්සමාජ පළමු ඛාණ්ඩ ශූරයන් වූ සිංහල ක්රීඩා සමාජ පිල ශ්රී ලංකව විවිධ අවධි වල නියෝජන කල ක්රීඩකයන් රැසක් සිටින එවක ශ්රී ලංකා නායක අර්ජුන රණතුංග සාමාන් ක්රීඩකයෙක් වසරේදී ක්රීඩා සමාජයට ක්රීඩා කර ඇත
10,4.0,1.41411E+18,“ක්‍රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන නෑ” සුනිල් ගුණවර්ධන https://t.co/jbNn6kMzEW #slathletics #namalrajapaksa @RajapaksaNamal https://t.co/iidrXsiHNe,7/11/2021 6:11,Politics,ක්රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන නෑ සුනිල් ගුණවර්ධන,ක්රීඩා ඇමැතිතුමා තවමත් හරි අය හරි තැන්වලට දාගෙන සුනිල් ගුණවර්ධන,ක්රීඩා ඇමැතිතුමා තවම හරි අය හරි තැන්වල දාගෙන සුනිල ගුණවර්ධන


In [43]:
#################################Linear regression classification #######################################

In [44]:
data['LABEL'].value_counts()

Other            96
Politics         54
Entertainment    17
Sport            16
Crime             2
Name: LABEL, dtype: int64

In [45]:
data.shape

(185, 8)

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
cv1 = CountVectorizer()
x = cv1.fit_transform(data['stemming_words'])

In [47]:
print(x.shape)

(185, 668)


In [48]:
x[0].toarray()

array([[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

In [49]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,data['LABEL'], test_size = 0.2,random_state = 42)

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score # mectric that use

In [51]:
#training
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [52]:
review = 'අන්තර්ජාලය ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජනය කිරීමේ කාන්තා'
commenttype = model.predict(cv1.transform([review]))[0]
if commenttype=='Crime':
    print('Crime Comment')
elif commenttype=='Sport':
     print('Sport Comment')
else:
    print('Other Comment')

Other Comment


In [53]:
###########################3 Naive bayes ###################333

In [68]:
data.loc[data['LABEL']=='Crime',"LABEL"] = 0
data.loc[data['LABEL']=='Entertainment',"LABEL"] = 1
data.loc[data['LABEL']=='Politics',"LABEL"] = 3
data.loc[data['LABEL']=='Sport',"LABEL"] = 4
data.loc[data['LABEL']=='Other',"LABEL"] = 5

In [73]:
data_x=data["stemming_words"]
data_y=data["LABEL"]

In [96]:
cv = TfidfVectorizer()

In [97]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=4)

In [98]:
x_train.head()

54                                                                                                                                                                  ඔබ පිදෙන අපගේ දෙවන කෙටි චිත්රපට සංඛාරා නුදුරේදී නරබන්න ඔබත් සූදානම් වන්න
144    පහත ජායාරූප වල සටහන් වැඩේ සම්පූර්ණයෙන් ආණ්ඩුවේ මඩ බළකා තුප්පැහි අශිෂ්ඨ වැඩකි නිදහස් අධ්යාපනය හඬ නඟන උන් දේශපාලන කඳවුර දේශපාලන මතට හොයන්න නොයා උන් පෙන්වන කාරනා දෙස බලනවා උන්ව දේශද්රෝහි කරන්න වගේ ප්රාථමික බංකොලොත් වැඩ කරන්නෙ ආණ්ඩුව
338                                                                       අපේ රටේ ත්රිවිධ හමුදාව කියන්නේ කාලයේ හිටපු කොළ කොටි කහා බළල්ලු එකී නොකී දේශපාලන මැරයෝ නොවෙයි බටහිර ජාතිකයෝ ලෝකය පෙන්වන්න හදන පැතිකඩ පණක් නැති ආයුධ අතට ගත්තු නුගත්
204                                                                                                                                                                                                                දවස කාලීන දේශපාලන වැඩසටහන
170                                                 

In [99]:
x_traincv=cv.fit_transform(x_train)

In [100]:
a = x_traincv.toarray()

In [101]:
a

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [102]:
len(a[0])

610

In [ ]:
###########  MultiNomial Naive bayes ############

In [108]:
mnb = MultinomialNB()
y_train = y_train.astype('int')

In [109]:
mnb.fit(x_traincv,y_train)

MultinomialNB()

In [110]:
x_testcv=cv.transform(x_test)

In [111]:
prediction = mnb.predict(x_testcv)

In [113]:
prediction

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [117]:
actual = np.array(y_test)

In [118]:
actual

array([3, 3, 3, 5, 5, 3, 4, 5, 5, 5, 3, 5, 5, 5, 1, 5, 5, 5, 5, 3, 5, 5,
       3, 5, 4, 4, 3, 4, 5, 5, 5, 3, 3, 3, 5, 3, 3], dtype=object)

In [121]:
count = 0
for i in range (len(prediction)):
    if prediction[i] == actual[i]:
        count = count +1

In [122]:
count #correct prediction

37

In [123]:
len(prediction) #total prediction

37

In [124]:
37/37

1.0